In [12]:
import os

def init_capture(capture_dir):
    # Create capture dir if it doesn't exist
    if not os.path.exists(capture_dir):
        os.makedirs(capture_dir)

def format_tabblock_url(state):
    # URL for FIPS and GNIS codes file -- https://www.census.gov/geo/reference/docs/state.txt'
    return 'ftp://ftp2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_{0}_tabblock10.zip'.format(state)

def format_lodes7_od_url(st, part, t, year):
    #http://lehd.ces.census.gov/data/lodes/LODES7/LODESTechDoc7.0.pdf
    return 'http://lehd.ces.census.gov/data/lodes/LODES7/{0}/od/{0}_od_{1}_{2}_{3}.csv.gz'.format(st, part, t, year)

def download_file(url, filename):
    command = "wget %s -O %s" % (url, filename)
    print "Downloading %s to %s" % (url, filename)
    !$command

def unzip_tabblock(filename, exdir):
    command = "unzip %s -d %s" % (filename, exdir)
    print "Unzip %s to %s" % (filename, exdir)
    !$command

def gunzip_lodes(filename):
    command = "gunzip %s" % filename
    print "Unzip %s" % filename
    !$command

ANSI_CODES = [
    ('01', 'al'), ('02', 'ak'), ('04', 'az'), ('05', 'ar'), ('06', 'ca'),
    ('08', 'co'), ('09', 'ct'), ('10', 'de'), ('11', 'dc'), ('12', 'fl'),
    ('13', 'ga'), ('15', 'hi'), ('16', 'id'), ('17', 'il'), ('18', 'in'),
    ('19', 'ia'), ('20', 'ks'), ('21', 'ky'), ('22', 'la'), ('23', 'me'),
    ('24', 'md'), ('25', 'ma'), ('26', 'mi'), ('27', 'mn'), ('28', 'ms'),
    ('29', 'mo'), ('30', 'mt'), ('31', 'ne'), ('32', 'nv'), ('33', 'nh'),
    ('34', 'nj'), ('35', 'nm'), ('36', 'ny'), ('37', 'nc'), ('38', 'nd'),
    ('39', 'oh'), ('40', 'ok'), ('41', 'or'), ('42', 'pa'), ('44', 'ri'),
    ('45', 'sc'), ('46', 'sd'), ('47', 'tn'), ('48', 'tx'), ('49', 'ut'),
    ('50', 'vt'), ('51', 'va'), ('53', 'wa'), ('54', 'wi'), ('55', 'wy'),
    ('56', 'wv')
]

In [ ]:
init_capture("capture/tabblock_2010")
capture_dir = "capture/tabblock_2010/"
pa_id = 42
init_capture(capture_dir)
pa_url = format_tabblock_url(pa_id)
pa_filename = capture_dir + os.path.basename(pa_url)
download_file(pa_url, pa_filename)
unzip_tabblock(pa_filename, pa_filename.split('.')[0])

In [ ]:
command = "createdb tl_2010_tabblock"
!$command
command = "psql -d tl_2010_tabblock -c 'CREATE EXTENSION postgis;'"
!$command

In [8]:
command = "shp2pgsql -s 4269:4326 capture/tabblock_2010/tl_2010_%s_tabblock10/tl_2010_%s_tabblock10.shp tl_2010_tabblock10 | psql -q -d tl_2010_tabblock" % (pa_id,pa_id)
!$command

Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]
                         addgeometrycolumn                          
--------------------------------------------------------------------
 public.tl_2010_tabblock10.geom SRID:4326 TYPE:MULTIPOLYGON DIMS:2 
(1 row)



In [9]:
command = "psql -d tl_2010_tabblock -c 'CREATE INDEX ON tl_2010_tabblock10 (geoid10);'"
!$command

CREATE INDEX


In [10]:
command = "psql -d tl_2010_tabblock -c 'CREATE INDEX ON tl_2010_tabblock10 USING GIST (geom);'"
!$command

CREATE INDEX


In [14]:
capture_dir = "capture/lodes7/pa/"
init_capture(capture_dir)
pa_url = format_lodes7_od_url('pa', 'main', 'JT00', '2011')
pa_filename = capture_dir + os.path.basename(pa_url)
download_file(pa_url, pa_filename) 
gunzip_lodes(pa_filename)


Unzip capture/lodes7/pa/pa_od_main_JT00_2011.csv.gz
